In [1]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [2]:
sp = SparkSession.builder.appName("Lookup").getOrCreate()
sp

22/10/05 23:40:43 WARN Utils: Your hostname, Harshitas-MacBook-Air-8.local resolves to a loopback address: 127.0.0.1; using 192.168.0.227 instead (on interface en0)
22/10/05 23:40:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 23:40:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from functools import reduce
from pyspark.sql import DataFrame

def read_tables(sp: SparkSession, file: str, ftype = "p", sample=False):
    """
    Helper function to read data from the desginated folder

    sp : Current SparkSession
    file : Type of data/Name of file name to be read
            (if file is "transactions" no ftype needed)
    ftype : File type (Parquet(p) or CSV(c))

    returns DataFrame read
    """
    # Root directory
    dir = "../data/tables/"
    
    # Transaction folders
    if file == "transactions":
        # Read all transactions together
        groups = [
            "transactions_20210228_20210827_snapshot/",
            "transactions_20210828_20220227_snapshot/",
            "transactions_20220228_20220828_snapshot/"
            ]

        # Read the different transaction folders
        final_list = []
        for g in groups:
            final_list.append(sp.read.option("inferSchema", True).parquet(dir + g))

        if not sample:
            return reduce(DataFrame.unionAll, final_list)
        return reduce(DataFrame.unionAll, final_list).sample(0.01)

    # Special file
    elif file == "tbl_consumer":
        return sp.read.option("inferSchema", True).option("header", True).option("delimiter", "|").csv("../data/tables/tbl_consumer.csv")

    # Parquet files
    if ftype == "p":
        return sp.read.option("inferSchema", True).parquet(dir + file + ".parquet")
    elif ftype == "c":
        return sp.read.option("inferSchema", True).option("header", True).csv(dir + file + ".csv")

In [4]:
trans = read_tables(sp, "transactions")
merchants_tbl = read_tables(sp, "tbl_merchants")
merchants = merchants_tbl.toPandas()

In [5]:
desc = sp.read.parquet("../data/tables/desc_avg.parquet")

In [6]:
unique_desc = desc.select('Description').distinct().toPandas()

In [7]:
unique_desc['desc_id'] = unique_desc.index
unique_desc

,Description,desc_id
0,lawn garden supply outlet including nursery,0
1,bicycle shop sale service,1
2,antique shop sale repair restoration service,2
3,book periodical newspaper,3
4,artist supply craft shop,4
5,gift card novelty souvenir shop,5
6,shoe shop,6
7,health beauty spa,7
8,art dealer gallery,8
9,music shop musical instrument piano sheet music,9


In [8]:
unique_desc.to_csv("../data/tables/description_lookup_tbl.csv")